# Stock Data Scraping and cleaning (ETL)

“This project automates the extraction, validation, and transformation of sector-specific stock data from Yahoo Finance. It’s designed as the backend ETL pipeline for a future web app that helps beginner investors build a diversified portfolio. The code mimics real-world data workflows, including modular design, error handling, and reusability — with potential for extension to real-time APIs or cloud data sources.”

The code is to show how to scrap stock data through yfinance, merge and clean them

## List of companies

In [ ]:
import pandas as pd

# Example with 2 sectors
data = [
    # Technology sector
    {"Sector": "Technology", "Ticker": "AAPL", "Company Name": "Apple Inc."},
    {"Sector": "Technology", "Ticker": "MSFT", "Company Name": "Microsoft Corporation"},
    {"Sector": "Technology", "Ticker": "GOOGL", "Company Name": "Alphabet Inc."},
    {"Sector": "Technology", "Ticker": "NVDA", "Company Name": "NVIDIA Corporation"},
    {"Sector": "Technology", "Ticker": "ADBE", "Company Name": "Adobe Inc."},

    # Healthcare sector
    {"Sector": "Healthcare", "Ticker": "PFE", "Company Name": "Pfizer Inc."},
    {"Sector": "Healthcare", "Ticker": "JNJ", "Company Name": "Johnson & Johnson"},
    {"Sector": "Healthcare", "Ticker": "LLY", "Company Name": "Eli Lilly and Company"},
    {"Sector": "Healthcare", "Ticker": "MRK", "Company Name": "Merck & Co., Inc."},
    {"Sector": "Healthcare", "Ticker": "ABT", "Company Name": "Abbott Laboratories"}
    
    # Add more names if you'd like
]

# Convert to DataFrame and save
df = pd.DataFrame(data)
output_path = "/mnt/data/stock_sector_list_example.csv"
df.to_csv(output_path, index=False)

output_path

## List of Tickers

In [ ]:

# List of all my tickers
tickers = [
    #Technology
    'AAPL', 'MSFT', 'AMD', 'ADBE', 'AVGO', 'INTC', 'CRM', 'CSCO', 'NVDA', 'ORCL', 
    'QCOM', 'PLTR', 'IBM', 'NOW', 'TXN', 'CDNS', 'ASML', 'GFS', 'APP', 'ANSS', 
    
    #Healthcare
    'ABBV', 'ABT', 'AMGN', 'JNJ', 'PFE', 'LLY', 'MRK', 'GEHC', 'ISRG', 'MDT', 
    'DHR', 'GILD', 'BIIB', 'DXCM', 'IDXX', 'BMY', 'TMO', 'AZN', 'HCA', 'CVS', 
    
    #Financials
    'AIG', 'AXP', 'BAC', 'BK', 'C', 'GS', 'JPM', 'MA', 'MET', 'MS', 
    'PYPL', 'SCHW', 'USB', 'V', 'BRK.B', 'COF', 'BLK', 'TROW', 'PGR', 'FITB', 
    
    #Consumer_Cyclical
    'AMZN', 'TSLA', 'HD', 'MCD', 'BKNG', 'GM', 'LOW', 'NKE', 'SBUX', 'TGT',
    'ABNB', 'YUM', 'ROST', 'TJX', 'ULTA', 'F', 'EBAY', 'ETSY', 'BBY', 
    
    #Consumer_Defensive
    'CL', 'COST', 'KO', 'MDLZ', 'MO', 'PG', 'PEP', 'WMT', 'KDP', 'CLX', 
    'KR', 'UNFI', 'GIS', 'TSN', 'SYY', 'ADM', 'KMB', 'EL', 'HSY', 'CHD',
    
    #Energy
    'COP', 'CVX', 'XOM', 'FANG', 'BKR', 'EOG', 'OXY', 'PSX', 'HAL', 'SLB', 
    'VLO', 'HES', 'DVN', 'MPC', 'APA', 'CPE', 'SM', 'MTDR', 'PDCE',  'PXD',
    
    #Industrials
    'BA', 'CAT', 'DE', 'EMR', 'FDX', 'GD', 'HON', 'LMT', 'UNP', 'UPS', 
    'CSX', 'ADP', 'MMM', 'RTX', 'GE', 'SWK', 'ITW', 'ETN', 'IR', 'PCAR', 
    
    #Communication Services
    'DIS','GOOG', 'GOOGL', 'META', 'CMCSA', 'T', 'TMUS', 'CHTR', 'NFLX', 'VZ', 
    'SIRI', 'PARA', 'FOXA', 'WBD', 'TTWO', 'ATVI', 'LYV', 'BIDU', 'NTES', 'SPOT', 
    
    #Utilities
    'DUK', 'NEE', 'SO', 'EXC', 'CEG', 'XEL', 'AEP', 'ES', 'D', 'NRG', 
    'PPL', 'PEG', 'ED', 'EVRG', 'EIX', 'WEC', 'AWK', 'ATO', 'SRE', 'CMS', 
    
    #Real_Estate
    'AMT', 'CSGP', 'SPG', 'WELL', 'O', 'PLD', 'BXP', 'EQIX', 'PSA', 'EQR', 
    'VNO', 'SLG', 'AVB', 'FRT', 'OHI', 'DLR', 'HST', 'WY', 'IRM', 'ARE', 
    
    #Basic_Materials
    'LIN', 'DD', 'ECL', 'FCX', 'NEM', 'APD', 'MOS', 'PPG', 'RPM', 'CE', 
    'EMN', 'ALB', 'VMC', 'CF', 'TREX', 'MLM', 'IFF', 'NUE', 'AVY', 'BALL']


### 1. Fetch Meta info of all companies (sector by sector)

This script allows you to fetch companies' meta info, such as sector, industry, marketcap, dividend yield, etc... you can fetch all the tickers at the same time, or sector by sector and save them individually to your local disk so that you can merge the data with the day trading data (see next script) to do analysis from there. 

In [ ]:
import yfinance as yf
import pandas as pd
import os
    
tickers = ['AMZN', 'TSLA', 'HD', 'MCD', 'BKNG', 'GM', 'LOW', 'NKE', 'SBUX', 'TGT', 
    'DIS', 'ABNB', 'YUM', 'ROST', 'TJX', 'ULTA', 'F', 'EBAY', 'ETSY', 'BBY']
metadata_list = []

for ticker in tickers:
    print(f"Fetching metadata for {ticker}...")
    try:
        t = yf.Ticker(ticker)
        info = t.info
        selected_info = {
            "ticker": ticker,
            "sector": info.get("sector"),
            "industry": info.get("industry"),
            "marketCap": info.get("marketCap"),
            "beta": info.get("beta"),
            "dividendYield": info.get("dividendYield"),
            "trailingPE": info.get("trailingPE"),
            "forwardPE": info.get("forwardPE"),
            "earningsQuarterlyGrowth": info.get("earningsQuarterlyGrowth"),
            "fullTimeEmployees": info.get("fullTimeEmployees"),
            "country": info.get("country"),
            "website": info.get("website")
        }
        metadata_list.append(selected_info)
    except Exception as e:
        print(f"Error retrieving data for {ticker}: {e}")

df = pd.DataFrame(metadata_list)

# Save to specified path
output_path = r"/Users/xiejing/Desktop/Codeoptest/Personal_Project/Consumer_Discretionary/Consumer_Discretionary_metadata.csv"
os.makedirs(os.path.dirname(output_path), exist_ok=True)
df.to_csv(output_path, index=False)
print(f"Saved to {output_path}")

Fetching metadata for AMZN...
Fetching metadata for TSLA...
Fetching metadata for HD...
Fetching metadata for MCD...
Fetching metadata for BKNG...
Fetching metadata for GM...
Fetching metadata for LOW...
Fetching metadata for NKE...
Fetching metadata for SBUX...
Fetching metadata for TGT...
Fetching metadata for DIS...
Fetching metadata for ABNB...
Fetching metadata for YUM...
Fetching metadata for ROST...
Fetching metadata for TJX...
Fetching metadata for ULTA...
Fetching metadata for F...
Fetching metadata for EBAY...
Fetching metadata for ETSY...
Fetching metadata for BBY...
Saved to /Users/xiejing/Desktop/Codeoptest/Personal_Project/Consumer_Discretionary/Consumer_Discretionary_metadata.csv


### 2. Day Trading Info

In [22]:
import yfinance as yf
import os

tickers = ['LIN', 'DD', 'ECL', 'FCX', 'NEM', 'APD', 'MOS', 'PPG', 'RPM', 'CE', 
    'EMN', 'ALB', 'VMC', 'CF', 'TREX', 'MLM', 'IFF', 'NUE', 'AVY', 'BALL']
save_path = r"/Users/xiejing/Desktop/Codeoptest/Personal_Project/Materials"

os.makedirs(save_path, exist_ok=True)

for ticker in tickers:
    print(f"Downloading {ticker}...")
    try:
        df = yf.download(ticker, start="2020-01-01", end="2025-05-20")
        if not df.empty:
            df.to_csv(os.path.join(save_path, f"{ticker}.csv"))
            print(f"Saved: {ticker}.csv")
        else:
            print(f"No data for {ticker}")
    except Exception as e:
        print(f"Error downloading {ticker}: {e}")

[*********************100%***********************]  1 of 1 completed


Saved: LIN.csv


[*********************100%***********************]  1 of 1 completed


Saved: DD.csv


[*********************100%***********************]  1 of 1 completed


...

Saved: NUE.csv


[*********************100%***********************]  1 of 1 completed


Saved: AVY.csv


[*********************100%***********************]  1 of 1 completed

Saved: BALL.csv


### 3. Merge the trading data (11 sectors)

In [23]:
import pandas as pd
import os

tickers = ['LIN', 'DD', 'ECL', 'FCX', 'NEM', 'APD', 'MOS', 'PPG', 'RPM', 'CE', 
    'EMN', 'ALB', 'VMC', 'CF', 'TREX', 'MLM', 'IFF', 'NUE', 'AVY', 'BALL']
folder_path = r"/Users/xiejing/Desktop/Codeoptest/Personal_Project/Materials"
combined = []

for ticker in tickers:
    file_path = os.path.join(folder_path, f"{ticker}.csv")
    try:
        df = pd.read_csv(file_path)
        df['Ticker'] = ticker
        combined.append(df)
    except Exception as e:
        print(f"Failed to read {file_path}: {e}")

if combined:
    merged_df = pd.concat(combined, ignore_index=True)
    output_path = os.path.join(folder_path, "20_Materials_Companies_nasdq_trading_data_2020.01-2025.05.csv")
    merged_df.to_csv(output_path, index=False)
    print(f"Saved merged CSV to {output_path}")
else:
    print("No data files found or loaded.")


Saved merged CSV to /Users/xiejing/Desktop/Codeoptest/Personal_Project/Materials/20_Materials_Companies_nasdq_trading_data_2020.01-2025.05.csv


### 4. Merge the Meta infos

In [24]:
import pandas as pd
import os

# Folder with all metadata CSVs
folder_path = "/Users/xiejing/Desktop/Codeoptest/Investor_Portfolio_Recommender_for_Beginners"

# Load and concatenate
metadata_files = [f for f in os.listdir(folder_path) if "metadata" in f.lower()]
meta_list = []

for file in metadata_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    meta_list.append(df)

# Combine all into one DataFrame
meta_all = pd.concat(meta_list, ignore_index=True)

# Save final metadata
meta_all.to_csv(os.path.join(folder_path, "merged_metadata.csv"), index=False)
print("✅ All metadata files merged.")


✅ All metadata files merged.


### 5. Clean the metadata file

In [2]:
import pandas as pd
# Read File
metadata = pd.read_csv('/Users/xiejing/Desktop/Codeoptest/Investor_Portfolio_Recommender_for_Beginners/merged_metadata.csv')
metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198 entries, 0 to 197
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ticker                   198 non-null    object 
 1   sector                   195 non-null    object 
 2   industry                 195 non-null    object 
 3   marketCap                195 non-null    float64
 4   beta                     195 non-null    float64
 5   dividendYield            172 non-null    float64
 6   trailingPE               185 non-null    float64
 7   forwardPE                195 non-null    float64
 8   earningsQuarterlyGrowth  175 non-null    float64
 9   fullTimeEmployees        195 non-null    float64
 10  country                  195 non-null    object 
 11  website                  195 non-null    object 
dtypes: float64(7), object(5)
memory usage: 18.7+ KB


In [3]:
# check if value is null in any column

metadata.isnull().sum()

ticker                      0
sector                      3
industry                    3
marketCap                   3
beta                        3
dividendYield              26
trailingPE                 13
forwardPE                   3
earningsQuarterlyGrowth    23
fullTimeEmployees           3
country                     3
website                     3
dtype: int64

In [11]:
# Drop rows where all columns except 'ticker' are NaN
metadata_cleaned = metadata.dropna(subset=metadata.columns.difference(['ticker']), how='all')

# Reset the index after dropping
metadata_cleaned = metadata_cleaned.reset_index(drop=True)

# List of columns to impute with "Nah"
cols_to_fill = [
    "sector", "industry", "marketCap", "beta", "dividendYield",
    "trailingPE", "forwardPE", "earningsQuarterlyGrowth",
    "fullTimeEmployees", "country", "website"
]


# Replace NaNs with "Nah" in those columns
metadata_cleaned[cols_to_fill] = metadata_cleaned[cols_to_fill].fillna("NaN")

metadata_cleaned['dividendYield'] = pd.to_numeric(metadata_cleaned['dividendYield'], errors='coerce')
metadata_cleaned['trailingPE'] = pd.to_numeric(metadata_cleaned['trailingPE'], errors='coerce')

In [5]:
# check again if value is null in any column
metadata_cleaned.isnull().sum()

ticker                     0
sector                     0
industry                   0
marketCap                  0
beta                       0
dividendYield              0
trailingPE                 0
forwardPE                  0
earningsQuarterlyGrowth    0
fullTimeEmployees          0
country                    0
website                    0
dtype: int64

### 6. Merge the Trading data file

In [36]:
# Folder with all metadata CSVs
folder_path = "/Users/xiejing/Desktop/Codeoptest/Investor_Portfolio_Recommender_for_Beginners"

# Load and concatenate
trading_files = [f for f in os.listdir(folder_path) if "trading" in f.lower()]
trading_list = []

for file in trading_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    trading_list.append(df)

# Combine all into one DataFrame
trading_all = pd.concat(trading_list, ignore_index=True)

# Save final metadata
trading_all.to_csv(os.path.join(folder_path, "merged_tradingdata.csv"), index=False)
print("✅ All tradingdata files merged.")

✅ All tradingdata files merged.


### 7. Clean the Trading data file

In [6]:
import pandas as pd
# Read File
tradingdata = pd.read_csv('/Users/xiejing/Desktop/Codeoptest/Investor_Portfolio_Recommender_for_Beginners/merged_tradingdata.csv')
tradingdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 288640 entries, 0 to 288639
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   Price   288640 non-null  object
 1   Close   288425 non-null  object
 2   High    288425 non-null  object
 3   Low     288425 non-null  object
 4   Open    288425 non-null  object
 5   Volume  288425 non-null  object
 6   Ticker  288640 non-null  object
dtypes: object(7)
memory usage: 15.4+ MB


In [7]:
tradingdata.isnull().sum()

Price       0
Close     215
High      215
Low       215
Open      215
Volume    215
Ticker      0
dtype: int64

In [8]:
# some injected partial header rows are detected and need for removal

# Step 1: Remove rows where 'Price' contains 'Ticker' or other header keywords
header_keywords = ['Ticker', 'Date', 'Close', 'High', 'Low', 'Open', 'Volume']

# Keep rows where 'Price' is not in the header keywords
tradingdata_cleaned = tradingdata[~tradingdata['Price'].isin(header_keywords)].copy()

# Step 2: Convert 'Price' to datetime and rename to 'Date'
tradingdata_cleaned['Price'] = pd.to_datetime(tradingdata_cleaned['Price'], errors='coerce')
tradingdata_cleaned.rename(columns={'Price': 'Date'}, inplace=True)

# Step 3: Drop rows with invalid or missing Date
tradingdata_cleaned = tradingdata_cleaned.dropna(subset=['Date'])

# Optional: Convert numeric columns to correct types
numeric_cols = ['Close', 'High', 'Low', 'Open', 'Volume']
for col in numeric_cols:
    if col in tradingdata_cleaned.columns:
        tradingdata_cleaned[col] = pd.to_numeric(tradingdata_cleaned[col], errors='coerce')

# Step 4: Reset index
tradingdata_cleaned = tradingdata_cleaned.reset_index(drop=True)

In [45]:
print(tradingdata_cleaned.head(), tradingdata_cleaned.tail())

        Date      Close       High        Low       Open   Volume Ticker
0 2020-01-02  45.015629  45.111296  44.563388  44.911266  3104600    AIG
1 2020-01-03  44.667755  44.763423  44.302480  44.554693  2358800    AIG
2 2020-01-06  44.702541  44.911267  44.389449  44.467722  2699700    AIG
3 2020-01-07  44.450325  44.728628  44.111143  44.659051  4580100    AIG
4 2020-01-08  44.972137  45.441776  44.450319  44.450319  4832100    AIG              Date      Close       High        Low       Open   Volume Ticker
288205 2025-05-13  93.802017  94.629391  92.825125  94.539672  3675500    CHD
288206 2025-05-14  92.865005  93.532882  91.758520  93.363422  2539100    CHD
288207 2025-05-15  94.559998  94.639999  92.610001  93.250000  1908300    CHD
288208 2025-05-16  95.820000  95.940002  94.250000  94.639999  2058100    CHD
288209 2025-05-19  95.970001  96.220001  95.339996  95.820000  2250000    CHD


In [ ]:
# check if value is null in any column
tradingdata_cleaned.isnull().sum()

Date      0
Close     0
High      0
Low       0
Open      0
Volume    0
Ticker    0
dtype: int64

In [12]:
# Export the cleaned trading data
tradingdata_cleaned.to_csv("final_cleaned_trading_data.csv", index=False)

# Export the cleaned metadata
metadata_cleaned.to_csv("final_cleaned_metadata.csv", index=False)

### 8. Unify columns' naming style in three files

In [45]:
## The sector_date

# read sector_data file and change all columns' name in lower case
sector_data = pd.read_csv('/Users/xiejing/Desktop/Codeoptest/Investor_Portfolio_Recommender_for_Beginners/stock_sector_list_2025.csv')
sector_data.columns = [col.lower() for col in sector_data.columns]

# Rename a column for easier analysis later
sector_data.rename(columns={'company name': 'company_name'}, inplace=True)
sector_data['sector'] = sector_data['sector'].replace('conmmunication_services', 'Conmmunication_Services')
sector_data['sector'] = sector_data['sector'].replace('real_estate', 'Real_Estate')
sector_data['sector'] = sector_data['sector'].replace('Financials', 'Financial_Services')

# Save the updated dataset
sector_data.to_csv('/Users/xiejing/Desktop/Codeoptest/Investor_Portfolio_Recommender_for_Beginners/stock_sector_list_2025.csv', index=False)

In [43]:
## The meta_date

# read metadata file and change all columns' name in lower case
meta_data = pd.read_csv('/Users/xiejing/Desktop/Codeoptest/Investor_Portfolio_Recommender_for_Beginners/final_cleaned_metadata.csv')


# Rename a column for easier analysis later
meta_data['sector'] = meta_data['sector'].replace('basic_materials', 'Basic_Materials')
meta_data['sector'] = meta_data['sector'].replace('consumer_cyclical', 'Consumer_Cyclical')
meta_data['sector'] = meta_data['sector'].replace('consumer_defensive', 'Consumer_Defensive')

meta_data['sector'] = meta_data['sector'].replace('real_estate', 'Real_Estate')
meta_data['sector'] = meta_data['sector'].replace('Financial Services', 'Financial_Services')


meta_data.loc[meta_data['ticker'].isin(['AVY', 'BALL']), 'sector'] = 'Basic_Materials'

In [64]:
# Step 1: Read the file
meta_data = pd.read_csv('/Users/xiejing/Desktop/Codeoptest/Investor_Portfolio_Recommender_for_Beginners/final_cleaned_metadata.csv')

# Step 2: Optional value correction (if needed)
meta_data['sector'] = meta_data['sector'].replace('conmmunication_services', 'Communication_Services')

# Step 3: Save as comma-separated CSV (new file)
meta_data.to_csv('/Users/xiejing/Desktop/Codeoptest/Investor_Portfolio_Recommender_for_Beginners/final_cleaned_metadata.csv', index=False)


In [66]:
meta_data.columns = [col.strip().lower().replace(' ', '_') for col in meta_data.columns]
meta_data.columns = meta_data.columns.str.replace('marketcap', 'market_cap') \
                                     .str.replace('dividendyield', 'dividend_yield') \
                                     .str.replace('trailingpe', 'trailing_pe') \
                                     .str.replace('forwardpe', 'forward_pe') \
                                     .str.replace('earningsquarterlygrowth', 'earnings_quarterly_growth') \
                                     .str.replace('fulltimeemployees', 'full_time_employees')


meta_data.to_csv('/Users/xiejing/Desktop/Codeoptest/Investor_Portfolio_Recommender_for_Beginners/final_cleaned_metadata.csv', index=False)


In [65]:
meta_data['sector'].value_counts() 

sector
Technology                21
Consumer_Defensive        21
Industrials               20
Real_Estate               20
Healthcare                20
Utilities                 20
Basic_Materials           19
Communication_Services    19
Consumer_Cyclical         18
Financial_Services        18
Energy                    17
Name: count, dtype: int64

In [69]:
## The trading_date

# read trading_data file and change all columns' name in lower case
trading_data = pd.read_csv('/Users/xiejing/Desktop/Codeoptest/Investor_Portfolio_Recommender_for_Beginners/final_cleaned_trading_data.csv')
trading_data.columns = [col.lower() for col in trading_data.columns]

trading_data.to_csv('/Users/xiejing/Desktop/Codeoptest/Investor_Portfolio_Recommender_for_Beginners/final_cleaned_trading_data.csv', index=False)


### Check Discrepencies in ticker in three datasets

In [ ]:
# Get sets of tickers from each dataset
meta_tickers = set(meta_data['ticker'].dropna().unique())
trading_tickers = set(trading_data['ticker'].dropna().unique())
sector_tickers = set(sector_data['ticker'].dropna().unique())

# Find discrepancies
only_in_meta = meta_tickers - trading_tickers - sector_tickers
only_in_trading = trading_tickers - meta_tickers - sector_tickers
only_in_sector = sector_tickers - meta_tickers - trading_tickers

# Show results
print("Tickers only in meta_data:", only_in_meta)
print("Tickers only in trading_data:", only_in_trading)
print("Tickers only in sector_data:", only_in_sector)

# Optional: tickers common to all
common_all = meta_tickers & trading_tickers & sector_tickers
print("Tickers common to all three datasets:", common_all)


In [ ]:
# Remove tickers that are only in sector_data and not in the others
sector_data_cleaned = sector_data[~sector_data['ticker'].isin(only_in_sector)]
sector_data_cleaned

sector_data_cleaned.to_csv('/Users/xiejing/Desktop/Codeoptest/Investor_Portfolio_Recommender_for_Beginners/stock_sector_list_2025.csv', index=False)

In [91]:
sector_cleaned = pd.read_csv('/Users/xiejing/Desktop/Codeoptest/Investor_Portfolio_Recommender_for_Beginners/stock_sector_list_2025.csv')

# Get unique tickers from each dataset
sector_cleaned_tickers = set(sector_cleaned['ticker'].dropna().unique())
meta_tickers = set(meta_data['ticker'].dropna().unique())

# Find differences
only_in_sector = sector_cleaned_tickers - meta_tickers
only_in_meta = meta_tickers - sector_cleaned_tickers
common_tickers = sector_cleaned_tickers & meta_tickers

# Print results
print("Tickers only in sector_data_cleaned:", only_in_sector)
print("Tickers only in meta_data:", only_in_meta)
print("Tickers in both:", common_tickers)
print("Counts — sector:", len(sector_cleaned_tickers), "meta:", len(meta_tickers), "common:", len(common_tickers))


Tickers only in sector_data_cleaned: {'V'}
Tickers only in meta_data: set()
Tickers in both: {'XEL', 'CSGP', 'GE', 'VNO', 'EMN', 'CLX', 'VZ', 'MDT', 'ARE', 'ETN', 'BBY', 'DHR', 'BAC', 'BALL', 'HD', 'YUM', 'MSFT', 'ED', 'PSA', 'DXCM', 'CMCSA', 'CSX', 'ABNB', 'JPM', 'NVDA', 'MS', 'OXY', 'BMY', 'D', 'PGR', 'BIIB', 'KO', 'CHTR', 'MET', 'DIS', 'GILD', 'BIDU', 'SBUX', 'TJX', 'TREX', 'MCD', 'XOM', 'DLR', 'CL', 'EQR', 'MRK', 'LOW', 'NKE', 'ASML', 'HES', 'TMO', 'CF', 'MPC', 'LIN', 'AWK', 'PPL', 'CRM', 'NTES', 'LLY', 'FITB', 'TSLA', 'SRE', 'WMT', 'PG', 'HSY', 'AMD', 'GFS', 'AVGO', 'ABT', 'BXP', 'UNFI', 'GM', 'SM', 'AZN', 'ROST', 'PLTR', 'RTX', 'MTDR', 'PPG', 'NUE', 'EVRG', 'IRM', 'NFLX', 'BKNG', 'EMR', 'UPS', 'ADBE', 'ULTA', 'EQIX', 'FCX', 'HST', 'PEP', 'NRG', 'CEG', 'AIG', 'SLB', 'SLG', 'SIRI', 'BLK', 'ANSS', 'TXN', 'BA', 'EBAY', 'EXC', 'ADP', 'UNP', 'MO', 'ORCL', 'SPG', 'BKR', 'IBM', 'APP', 'ISRG', 'O', 'FDX', 'ATO', 'AVB', 'GEHC', 'AMZN', 'PARA', 'LYV', 'EL', 'TGT', 'PLD', 'ABBV', 'GOOGL', 'W

In [92]:
duplicate_tickers = sector_cleaned['ticker'][sector_cleaned['ticker'].duplicated(keep=False)]
print(sector_cleaned[sector_cleaned['ticker'].isin(duplicate_tickers)])


                      sector ticker company_name
69         Consumer_Cyclical    DIS       Disney
136  Conmmunication_Services    DIS  Walt Disney


In [95]:
# Remove the row where ticker is 'DIS' and sector is 'Consumer_Cyclical'
sector_cleaned = sector_cleaned[~((sector_cleaned['ticker'] == 'DIS') & (sector_cleaned['sector'] == 'Consumer_Cyclical'))]
sector_cleaned = sector_cleaned[~(sector_cleaned['ticker'] == 'V')]

sector_cleaned.to_csv('/Users/xiejing/Desktop/Codeoptest/Investor_Portfolio_Recommender_for_Beginners/stock_sector_list_2025.csv', index=False)

In [96]:
# Delete duplicated rows in trading data

# Step 1: Filter only DIS
dis_data = trading_data[trading_data['ticker'] == 'DIS']

# Step 2: Drop duplicate dates, keeping the last one
dis_data_cleaned = dis_data.drop_duplicates(subset='date', keep='last')

# Step 3: Replace DIS rows in the original dataset with cleaned version
trading_data_no_dis = trading_data[trading_data['ticker'] != 'DIS']
trading_data_final = pd.concat([trading_data_no_dis, dis_data_cleaned], ignore_index=True)

# Optional: Save the cleaned dataset
trading_data_final.to_csv('/Users/xiejing/Desktop/Codeoptest/Investor_Portfolio_Recommender_for_Beginners/final_cleaned_trading_data.csv', index=False)

In [ ]:
# Check for duplicates based on ticker + date
duplicates = trading_data_final.duplicated(subset=['ticker', 'date'], keep=False)

# Show all duplicate rows (if any)
duplicate_rows = trading_data_final[duplicates]
print(duplicate_rows)
